## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## 2) Prepare the data for processing

In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
cd ~/Data/challenge/

/home/dsc/Data/challenge


In [19]:
b = pd.read_csv('bookings.csv.bz2',sep='^', nrows=1200000)

In [20]:
b.shape

(1200000, 38)

In [21]:
b.drop_duplicates().shape

(1000000, 38)

In [41]:
%%time
bc = pd.read_csv('bookings.csv.bz2',sep='^', chunksize=300000, low_memory=False)

all_chunks=pd.DataFrame()
for i, chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks=all_chunks.drop_duplicates()
    print (i, len(all_chunks))


0 300000
1 600000
2 900000
3 1000000
4 1000000
5 1000000
6 1000000
7 1000000
8 1000000
9 1000000
10 1000000
11 1000000
12 1000000
13 1000000
14 1000000
15 1000000
16 1299993
17 1299993
18 1299993
19 1299993
20 1299993
21 1299993
22 1299993
23 1299993
24 1299993
25 1299993
26 1299993
27 1299993
28 1299993
29 1299993
30 1299993
31 1299993
32 1299993
33 1299993
CPU times: user 13min 2s, sys: 1min 2s, total: 14min 5s
Wall time: 14min 15s


In [49]:
all_chunks.to_csv('bookings_no_dup.csv', sep='^', index=False)

In [50]:
! bzip2 -f bookings_no_dup.csv
!ls -l

total 1099120
-rwxr-x--- 1 dsc dsc 554970628 mar 13  2018 bookings.csv.bz2
-rw-r--r-- 1 dsc dsc  72211641 ene 26 13:01 bookings_no_dup.csv.bz2
-rw-r--r-- 1 dsc dsc   4232732 ene 25 17:53 bookings_sample.csv
-rwxrwxrwx 1 dsc dsc    535893 ene 25 17:18 bookings_sample.csv.bz2
-rwxr-x--- 1 dsc dsc 483188920 mar 13  2018 searches.csv.bz2
-rw-r--r-- 1 dsc dsc  10013467 ene 26 12:22 searches_no_dup.csv.bz2
-rwxrwxrwx 1 dsc dsc    244720 ene 25 17:19 searches_sample.csv.bz2
-rw-r--r-- 1 dsc dsc     80140 ene 25 21:22 top_airports.csv


In [3]:
%%time
sc = pd.read_csv('searches.csv.bz2',sep='^', chunksize=300000, low_memory=False)

all_chunks=pd.DataFrame()
for i, chunk in enumerate(sc):
    all_chunks=all_chunks.append(chunk)
    all_chunks=all_chunks.drop_duplicates()
    print (i, len(all_chunks))

0 299999
1 358999
2 358999
3 358999
4 658992
5 718002
6 718002
7 718002
8 718002
9 718002
10 718002
11 718002
12 718002
13 718002
14 718002
15 718002
16 718002
17 718002
18 718002
19 718002
20 718002
21 718002
22 718002
23 718002
24 718002
25 718002
26 718002
27 718002
28 718002
29 718002
30 718002
31 718002
32 718002
33 718002
34 718002
35 718002
36 718002
37 718002
38 718002
39 718002
40 718002
41 718002
42 718002
43 718002
44 718002
45 718002
46 718002
47 718002
48 718002
49 718002
50 718002
51 718002
52 718002
53 718002
54 718002
55 718002
56 718002
57 718002
58 718002
59 718002
60 718002
61 718002
62 718002
63 718002
64 718002
65 718002
66 718002
67 718003
CPU times: user 23min 34s, sys: 1min 58s, total: 25min 32s
Wall time: 25min 38s


In [4]:
all_chunks.to_csv('searches_no_dup.csv', sep='^', index=False)

In [5]:
! bzip2 -f searches_no_dup.csv
!ls -l

total 1183904
-rwxr-x--- 1 dsc dsc 554970628 mar 13  2018 bookings.csv.bz2
-rw-r--r-- 1 dsc dsc  72211641 ene 26 13:01 bookings_no_dup.csv.bz2
-rw-r--r-- 1 dsc dsc   4232732 ene 25 17:53 bookings_sample.csv
-rwxrwxrwx 1 dsc dsc    535893 ene 25 17:18 bookings_sample.csv.bz2
-rwxr-x--- 1 dsc dsc 483188920 mar 13  2018 searches.csv.bz2
-rw-r--r-- 1 dsc dsc  17112287 ene 28 10:28 searches_no_dup.csv.bz2
-rwxrwxrwx 1 dsc dsc    244720 ene 25 17:19 searches_sample.csv.bz2
-rw-r--r-- 1 dsc dsc  79716511 ene 26 13:42 search_with_booking.csv
-rw-r--r-- 1 dsc dsc     80140 ene 25 21:22 top_airports.csv


### Action Plan

In [6]:
b = pd.read_csv('bookings_no_dup.csv.bz2',sep='^', usecols=['dep_port', 'arr_port', 'cre_date           '])

In [7]:
s=pd.read_csv('./searches_no_dup.csv.bz2', sep='^',usecols=['Destination', 'Origin', 'Date'])

In [8]:
b.shape, s.shape

((1299993, 3), (718003, 3))

In [9]:
s.reset_index(inplace=True)

In [10]:
s.head()

,index,Date,Origin,Destination
0,0,2013-01-01,TXL,AUH
1,1,2013-01-01,ATH,MIL
2,2,2013-01-01,ICT,SFO
3,3,2013-01-01,RNB,ARN
4,4,2013-01-01,OSL,MAD


In [11]:
s.dropna(inplace=True)

In [13]:
s.shape

(718002, 4)

we have lost 1 element!

In [17]:
b.columns=b.columns.map(lambda x:x.strip())
b['dep_port']=b['dep_port'].map(   lambda x:x.strip()   )
b['arr_port']=b['arr_port'].map(   lambda x:x.strip()   )

In [18]:
b['cre_date']=b['cre_date'].str[0:10]

In [19]:
b.head()

,cre_date,dep_port,arr_port
0,2013-02-22,ZRH,LHR
1,2013-03-26,SAL,CLT
2,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
4,2013-03-26,AKL,SVO


what do we have so far?

In [ ]:
#0 -->  add index column to searches, drop nan
#1-> make sure dep_port & arr_port dont have spaces
#2-> make sure cre_date and search_date have the same format

#--------- We are here

#3-> add Booked column with 1 in bookings
#4-> group by dep_port, arr_port, cre_date and sum Booked
#5-> left join searches + bookings adding Booked column
#6-> change Nan to 0 and if >0 a 1
#7-> add column in the file



b = pd.read_csv('bookings_no_dup.csv.bz2',sep='^', usecols=['dep_port', 'arr_port', 'cre_date           '])
s=pd.read_csv('./searches_no_dup.csv.bz2', sep='^',usecols=['Destination', 'Origin', 'Date'])

s.reset_index(inplace=True)
s.dropna(inplace=True)

b.columns=b.columns.map(lambda x:x.strip())
b['dep_port']=b['dep_port'].map(   lambda x:x.strip()   )
b['arr_port']=b['arr_port'].map(   lambda x:x.strip()   )
b['cre_date']=b['cre_date'].str[0:10]

In [20]:
b['Booked']=1

In [21]:
b_gr=b.groupby(['dep_port', 'arr_port', 'cre_date']).sum().reset_index()

In [22]:
b_gr.head()

,dep_port,arr_port,cre_date,Booked
0,AAB,ROV,2013-05-29,2
1,AAB,ROV,2013-08-24,2
2,AAE,ALG,2013-01-07,1
3,AAE,ALG,2013-01-08,4
4,AAE,ALG,2013-01-09,4


In [ ]:
#0 -->  add index column to searches, drop nan
#1-> make sure dep_port & arr_port dont have spaces
#2-> make sure cre_date and search_date have the same format
#3-> add Booked column with 1 in bookings
#4-> group by dep_port, arr_port, cre_date and sum Booked

#------- we are here

#5-> left join searches + bookings adding Booked column
#6-> change Nan to 0 and if >0 a 1
#7-> add column in the file



b = pd.read_csv('bookings_no_dup.csv.bz2',sep='^', usecols=['dep_port', 'arr_port', 'cre_date           '])
s=pd.read_csv('./searches_no_dup.csv.bz2', sep='^',usecols=['Destination', 'Origin', 'Date'])

s.reset_index(inplace=True)
s.dropna(inplace=True)

b.columns=b.columns.map(lambda x:x.strip())
b['dep_port']=b['dep_port'].map(   lambda x:x.strip()   )
b['arr_port']=b['arr_port'].map(   lambda x:x.strip()   )
b['cre_date']=b['cre_date'].str[0:10]
b['Booked']=1
b_gr=b.groupby(['dep_port', 'arr_port', 'cre_date']).sum().reset_index()

In [23]:
s_b=s.merge(b_gr, how='left', left_on=['Destination', 'Origin', 'Date'], right_on=['arr_port', 'dep_port','cre_date'])

In [24]:
s_b.head()

,index,Date,Origin,Destination,dep_port,arr_port,cre_date,Booked
0,0,2013-01-01,TXL,AUH,NaN,NaN,NaN,NaN
1,1,2013-01-01,ATH,MIL,NaN,NaN,NaN,NaN
2,2,2013-01-01,ICT,SFO,NaN,NaN,NaN,NaN
3,3,2013-01-01,RNB,ARN,NaN,NaN,NaN,NaN
4,4,2013-01-01,OSL,MAD,NaN,NaN,NaN,NaN


In [25]:
s_b.drop(['arr_port', 'dep_port','cre_date'], axis=1, inplace=True)

In [26]:
s_b[ s_b['Booked']>0 ].head()

,index,Date,Origin,Destination,Booked
27,27,2013-01-01,RUH,JED,19.0
40,40,2013-01-01,DMM,MNL,5.0
59,59,2013-01-01,ATL,MIA,12.0
134,134,2013-01-01,MEL,SYD,12.0
172,172,2013-01-01,BOM,JED,4.0


In [27]:
s_b['Booked']=s_b['Booked'].map(lambda x: 1 if x>0 else 0)

In [28]:
s_b[ s_b['Booked']>0 ].head()

,index,Date,Origin,Destination,Booked
27,27,2013-01-01,RUH,JED,1
40,40,2013-01-01,DMM,MNL,1
59,59,2013-01-01,ATL,MIA,1
134,134,2013-01-01,MEL,SYD,1
172,172,2013-01-01,BOM,JED,1


In [29]:
s_b.head()

,index,Date,Origin,Destination,Booked
0,0,2013-01-01,TXL,AUH,0
1,1,2013-01-01,ATH,MIL,0
2,2,2013-01-01,ICT,SFO,0
3,3,2013-01-01,RNB,ARN,0
4,4,2013-01-01,OSL,MAD,0


In [ ]:
#0 -->  add index column to searches, drop nan
#1-> make sure dep_port & arr_port dont have spaces
#2-> make sure cre_date and search_date have the same format
#3-> add Booked column with 1 in bookings
#4-> group by dep_port, arr_port, cre_date and sum Booked
#5-> left join searches + bookings adding Booked column
#6-> change Nan to 0 and if >0 a 1
#------- we are here
#7-> add column in the file

b = pd.read_csv('bookings_no_dup.csv.bz2',sep='^', usecols=['dep_port', 'arr_port', 'cre_date           '])
s=pd.read_csv('./searches_no_dup.csv.bz2', sep='^',usecols=['Destination', 'Origin', 'Date'])

s.reset_index(inplace=True)
s.dropna(inplace=True)

b.columns=b.columns.map(lambda x:x.strip())
b['dep_port']=b['dep_port'].map(   lambda x:x.strip()   )
b['arr_port']=b['arr_port'].map(   lambda x:x.strip()   )
b['cre_date']=b['cre_date'].str[0:10]
b['Booked']=1
b_gr=b.groupby(['dep_port', 'arr_port', 'cre_date']).sum().reset_index()
s_b=s.merge(b_gr, how='left', left_on=['Destination', 'Origin', 'Date'], right_on=['arr_port', 'dep_port','cre_date'])
s_b.drop(['arr_port', 'dep_port','cre_date'], axis=1, inplace=True)
s_b['Booked']=s_b['Booked'].map(lambda x: 1 if x>0 else 0)

In [68]:
s_b.head()

,index,Date,Origin,Destination,Booked
0,0,2013-01-01,TXL,AUH,0
1,1,2013-01-01,ATH,MIL,0
2,2,2013-01-01,ICT,SFO,0
3,3,2013-01-01,RNB,ARN,0
4,4,2013-01-01,OSL,MAD,0


In [30]:
s_b.rename(columns={'index':'old_index_from_file'}, inplace=True)

In [31]:
s_b.head()

,old_index_from_file,Date,Origin,Destination,Booked
0,0,2013-01-01,TXL,AUH,0
1,1,2013-01-01,ATH,MIL,0
2,2,2013-01-01,ICT,SFO,0
3,3,2013-01-01,RNB,ARN,0
4,4,2013-01-01,OSL,MAD,0


In [33]:
s_b.tail() #we can see that index is different from the old index taken from the file
#this is due to 1 line which was dropped with dropna

,old_index_from_file,Date,Origin,Destination,Booked
717997,717998,2013-05-03,CPH,AYT,0
717998,717999,2013-05-03,NYC,YTO,0
717999,718000,2013-05-03,TRF,TRD,0
718000,718001,2013-05-03,IAH,RDU,0
718001,718002,2013-10-13,VIE,HA,0


We have to add booked column in the file without a duplicates. We can do this by using left join on the data frame of the whole file. The join should be done on index.

In [32]:
search_all=pd.read_csv('./searches_no_dup.csv.bz2', sep='^', low_memory=False)

In [34]:
search_all.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


In [35]:
result=search_all.merge(s_b,how='left', left_index=True, right_on='old_index_from_file')

In [36]:
result.head()

,Date_x,Time,TxnCode,OfficeID,Country,Origin_x,Destination_x,RoundTrip,NbSegments,Seg1Departure,...,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,old_index_from_file,Date_y,Origin_y,Destination_y,Booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0,2013-01-01,TXL,AUH,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,1,2013-01-01,ATH,MIL,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,2,2013-01-01,ICT,SFO,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,3,2013-01-01,RNB,ARN,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,4,2013-01-01,OSL,MAD,0.0


In [37]:
result.drop(['old_index_from_file', 'Date_y','Origin_y', 'Destination_y'], axis=1, inplace=True)

In [38]:
result.rename(columns={'Date_x':'Date', 'Origin_x':'Origin', 'Destination_x':'Destination'}, inplace=True)

In [39]:
result.to_csv('search_with_booking.csv', sep='^', index=False)

In [40]:
! head search_with_booking.csv

Date^Time^TxnCode^OfficeID^Country^Origin^Destination^RoundTrip^NbSegments^Seg1Departure^Seg1Arrival^Seg1Date^Seg1Carrier^Seg1BookingCode^Seg2Departure^Seg2Arrival^Seg2Date^Seg2Carrier^Seg2BookingCode^Seg3Departure^Seg3Arrival^Seg3Date^Seg3Carrier^Seg3BookingCode^Seg4Departure^Seg4Arrival^Seg4Date^Seg4Carrier^Seg4BookingCode^Seg5Departure^Seg5Arrival^Seg5Date^Seg5Carrier^Seg5BookingCode^Seg6Departure^Seg6Arrival^Seg6Date^Seg6Carrier^Seg6BookingCode^From^IsPublishedForNeg^IsFromInternet^IsFromVista^TerminalID^InternetOffice^Booked
2013-01-01^20:25:57^MPT^624d8c3ac0b3a7ca03e3c167e0f48327^DE^TXL^AUH^1.0^2.0^TXL^AUH^2013-01-26^D2^^AUH^TXL^2013-02-02^D2^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^FRA^0.0
2013-01-01^10:15:33^MPT^b0af35b31588dc4ab06d5cf2986e8e02^MD^ATH^MIL^0.0^1.0^ATH^MIL^2013-01-04^^^^^^^^^^^^^^^^^^^^^^^^^^^^1ASIWS^0^0^0^d41d8cd98f00b204e9800998ecf8427e^KIV^0.0
2013-01-01^18:04:49^MPT^3561a60621de06ab1badc8ca55699ef3^US^ICT^SFO^1.0^2.0^ICT^SFO^2013-08-